# Multi-Currency Spot Rate Consistency

This notebook describes how we can check that FX Spot Rates are consistent with each other, by checking that all cross rates can be accurately derived. The notebook does not take into account differing spot dates for the spot rate pairs, assuming that the effective date of the quoted rate is the maturity of the FX spot.

**Table of Contents:**
 - [1. Setting Up Variables](#1.-Setting-Up-Variables)
 - [2. Loading FX Spot Data](#2.-Loading-FX-Spot-Data)
 - [3. Spot Rate Consistency Checks](#3.-Spot-Rate-Consistency-Checks)


In [30]:
# Import generic non-LUSID packages
import os
import pandas as pd
from datetime import datetime, timedelta
import re
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
import json
import pytz
import numpy as np
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid
import lusid.models as lm
import lusid.api as la
from lusid.utilities import ApiClientFactory
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# Import key functions from Lusid-Python-Tools and other packages
from lusidjam import RefreshingToken

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.4f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path)

print ('LUSID Environment Initialised')
print ('LUSID API Version :', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version : 0.6.10002.0


# 1. Setting Up Variables

In [31]:
# Set required APIs
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
complex_market_data_api = api_factory.build(lusid.api.ComplexMarketDataApi)
aggregation_api = api_factory.build(lusid.api.AggregationApi)

In [32]:
# Define scopes
scope = "ibor"
market_data_scope = "ibor-multiccy"
market_supplier = "Lusid"
success_key = "Success"
failed_key = "Failed"

# 2. Loading FX Spot Data

In this notebook we load spot rates for multiple currency pairs to test the spot rate consistency.

In [33]:
# Read fx spot rates and make datetimes timezone aware
quotes_df = pd.read_csv("data/Multiccy-spotRates.csv")
quotes_df["Date"] = pd.to_datetime(quotes_df["Date"], dayfirst=True)
quotes_df["Date"] = quotes_df["Date"].apply(lambda x: x.replace(tzinfo=pytz.utc))
quotes_df

,Date,Rate,Pair
0,2022-09-01 00:00:00+00:00,1.0300,EUR/USD
1,2022-09-02 00:00:00+00:00,1.0200,EUR/USD
2,2022-09-05 00:00:00+00:00,1.0100,EUR/USD
3,2022-09-06 00:00:00+00:00,1.0000,EUR/USD
4,2022-09-07 00:00:00+00:00,0.9900,EUR/USD
5,2022-09-08 00:00:00+00:00,0.9800,EUR/USD
6,2022-09-09 00:00:00+00:00,0.9950,EUR/USD
7,2022-09-12 00:00:00+00:00,1.0050,EUR/USD
8,2022-09-01 00:00:00+00:00,1.2100,GBP/USD
9,2022-09-02 00:00:00+00:00,1.1900,GBP/USD


In [34]:
# Create quotes request
instrument_quotes = {
            index: lm.UpsertQuoteRequest(
            quote_id=lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider=market_supplier,
                    instrument_id=row["Pair"],
                    instrument_id_type="CurrencyPair",
                    quote_type="Rate",
                    field="mid",
                ),
                effective_at=row["Date"],
            ),
            metric_value=lm.MetricValue(value=row["Rate"], unit=row["Pair"]),
        )
    for index, row in quotes_df.iterrows()
}

# Upsert quotes into LUSID
response = quotes_api.upsert_quotes(
    scope=market_data_scope, request_body=instrument_quotes
)

if response.failed == {}:
    print(f"Quotes successfully loaded into LUSID. {len(response.values)} quotes loaded.")
else:
    print(f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID.")

Quotes successfully loaded into LUSID. 56 quotes loaded.


# 3. Spot Rate Consistency Checks

We set up 2 functions to obtain and calculate the FX rates.

 - calculate_cross_rate: This takes a currency pair, and a currency to cross through to calculate the derived FX spot rate
 
 - calculate_cross_rate: This takes a currency and a cross currency, and obtains the FX rate from the quote store

In [35]:
# function to calculate a cross rate
def calculate_cross_rate (cross, currency_pair, effective_at):
    
    return_response = dict()
    
    if currency_pair == f"{cross}/{cross}":
        return_response['value'] = 1
        return_response['status'] = success_key
        return return_response
    
    currencies = currency_pair.split("/")
    if len(currencies) != 2:
        print(f"Currency pair is not defined correctly: {currency_pair}")
        return_response['status'] =  failed_key
        return return_response

    base_ccy =currencies[0]
    quote_ccy = currencies[1]
    
    pair_base = f"{cross}/{cross}"
    pair_quote = f"{cross}/{cross}"
    rate_base = 1
    rate_quote = 1     
    
    if base_ccy == cross:
        response = get_spot_rate (cross, quote_ccy, effective_at)
        if response['status'] == success_key:
            pair_quote = response['unit']
            rate_quote = response['value']
        else:
            return_response['status'] = failed_key
            return return_response
    elif quote_ccy == cross:
        response = get_spot_rate (cross, base_ccy, effective_at)
        if response['status'] == success_key:
            pair_base = response['unit']
            rate_base = response['value']
        else:
            return_response['status'] = failed_key
            return return_response   
    else:
        # Crossing through the cross currency
        response = get_spot_rate (cross, base_ccy, effective_at)
        if response['status'] == success_key:
            pair_base = response['unit']
            rate_base = response['value']
        else:
            return_response['status'] = failed_key
            return return_response
        
        # Crossing through the cross currency
        response = get_spot_rate (cross, quote_ccy, effective_at)
        if response['status'] == success_key:
            pair_quote = response['unit']
            rate_quote = response['value']
        else:
            return_response['status'] = failed_key
            return return_response
    

    if pair_base[0:3] == base_ccy:
        if pair_quote[0:3] == quote_ccy:
            return_response['value'] = rate_base / rate_quote
        else:
            return_response['value'] = rate_base * rate_quote
    else:
        if pair_quote[0:3] == quote_ccy:
            return_response['value'] = (1 / rate_base) * (1 / rate_quote)
        else:
            return_response['value'] = rate_quote / rate_base   
    
    return_response['status'] = success_key
    return return_response

In [36]:
def get_spot_rate(cross, currency, effective_at):
     
    return_response = dict()
        
    quotes = {"Base" :  lm.QuoteSeriesId(
                provider=market_supplier,
                instrument_id=f"{currency}/{cross}",
                instrument_id_type="CurrencyPair",
                quote_type="Rate",
                field="mid"), 
            f"Quote" :  lm.QuoteSeriesId(
                provider=market_supplier,
                instrument_id=f"{cross}/{currency}",
                instrument_id_type="CurrencyPair",
                quote_type="Rate",
                field="mid")
             }
    
    quotes_response = quotes_api.get_quotes(
        scope = market_data_scope,
        effective_at = effective_at,
        request_body = quotes
    )
    
    try:
        key = list(quotes_response.values.keys())[0]  
        return_response['unit'] = quotes_response.values[key].metric_value.unit
        return_response['value'] = quotes_response.values[key].metric_value.value
        return_response['status'] = success_key
    except:
        return_response['status'] = failed_key
        
    return return_response

# 3.1 Test The Functions

In [37]:
quote_date = datetime(2022, 9, 2, tzinfo=pytz.utc)
response = get_spot_rate("USD", "JPY", quote_date)
display(response)
pair = response['unit']
rate = response['value']
print(f"{pair} : {rate}")

{'unit': 'USD/JPY', 'value': 134.0, 'status': 'Success'}

USD/JPY : 134.0


In [38]:
rate = calculate_cross_rate ("USD", "CHF/EUR", quote_date)
display(rate)

{'value': 1.0212418300653594, 'status': 'Success'}

In [39]:
rate = calculate_cross_rate ("GBP", "CHF/EUR", quote_date)
display(rate)

{'status': 'Failed'}

# 3.2 Checking across all FX Spot Quotes in the store

In [40]:
# Get all of the quotes in the quote store
fx_quotes = quotes_api.list_quotes_for_scope(
    scope = market_data_scope)

In [47]:
def check_quotes (cross_currency):
    print(f"\nChecking the quotes by crossing through {cross_currency}")
    
    for quote in fx_quotes.values:
        instrument = quote.quote_id.quote_series_id.instrument_id
        effective_at = quote.quote_id.effective_at
        calculated_rate = calculate_cross_rate(cross_currency, instrument, effective_at)
        if calculated_rate['status'] == failed_key:
            print(f"Could not get cross rates for {instrument}, effective at: {effective_at}")
        elif calculated_rate['value'] != quote.metric_value.value:
            print(f"Quotes are not matching for {instrument} at: {effective_at}. Calculated (through {cross_currency}): {calculated_rate['value']}, quote store: {quote.metric_value.value}")

In [49]:
# check the quotes crossing through the USD
check_quotes ("USD")

# now check the EUR crosses
check_quotes ("EUR")

# now check the EUR crosses
check_quotes ("GBP")



Checking the quotes by crossing through USD
Quotes are not matching for GBP/JPY at: 2022-09-09T00:00:00.0000000+00:00. Calculated (through USD): 163.67249999999999, quote store: 163.67
Quotes are not matching for GBP/JPY at: 2022-09-08T00:00:00.0000000+00:00. Calculated (through USD): 160.07999999999998, quote store: 160.08
Quotes are not matching for GBP/JPY at: 2022-09-07T00:00:00.0000000+00:00. Calculated (through USD): 157.54999999999998, quote store: 157.55
Quotes are not matching for GBP/JPY at: 2022-09-02T00:00:00.0000000+00:00. Calculated (through USD): 159.45999999999998, quote store: 159.46
Quotes are not matching for EUR/CHF at: 2022-09-12T00:00:00.0000000+00:00. Calculated (through USD): 0.9647999999999999, quote store: 0.96
Quotes are not matching for EUR/CHF at: 2022-09-09T00:00:00.0000000+00:00. Calculated (through USD): 0.96515, quote store: 0.97
Quotes are not matching for EUR/CHF at: 2022-09-08T00:00:00.0000000+00:00. Calculated (through USD): 0.9603999999999999, quo